Google mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys

drive_project_root = '/content/drive/MyDrive/#fastcampus'
sys.path.append(drive_project_root)

!ls

# Dataset EDA

### data
- Google 'oxford pet dataset'
- Download 'groundtruth data'

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')

In [ ]:
df = pd.read_csv('drive/MyDrive/data/annotations/list.txt', skiprows=6, delimiter=" ", header=None)
df.columns = ['file_name', 'id', 'species', 'breed']
df

### 숫자 분포 확인

In [ ]:
value_counts = df['species'].value_counts().sort_index()   # 1: 고양이, 2: 강아지

plt.bar(range(len(value_counts)), value_counts.values, align='center')
plt.xticks(range(len(value_counts)), value_counts.index.values)
plt.show()

In [ ]:
print(value_counts)
print('\nrange: ',range(len(value_counts)))
print('\nvalues: ',value_counts.values)

- 각 class에 대해 대략 200장 씩 이미지가 있다는 것 확인

In [ ]:
value_counts = df['id'].value_counts().sort_index()

plt.bar(range(len(value_counts)), value_counts.values, align='center')
plt.xticks(range(len(value_counts)), value_counts.index.values)
plt.tight_layout()    # x label이 겹치지 않게 출력
plt.show()

- 고양이와 개 각각에 대한 class 구성
- 고양이는 12가지, 강아지는 25가지 종류가 있음을 확인

In [ ]:
value_counts = df[df['species'] == 1]['breed'].value_counts().sort_index()  # 고양이만

plt.bar(range(len(value_counts)), value_counts.values, align='center')
plt.xticks(range(len(value_counts)), value_counts.index.values)
plt.tight_layout()    # x label이 겹치지 않게 출력
plt.show()

In [ ]:
value_counts = df[df['species'] == 2]['breed'].value_counts().sort_index()  # 강아지만

plt.bar(range(len(value_counts)), value_counts.values, align='center')
plt.xticks(range(len(value_counts)), value_counts.index.values)
plt.tight_layout()    # x label이 겹치지 않게 출력
plt.show()

- 파일들을 읽기 위한 library

In [ ]:
import os
from glob import glob

In [ ]:
image_dir = 'drive/MyDrive/data/images/'
bbox_dir = 'drive/MyDrive/data/annotations/xmls/'  # bounding box
seg_dir = 'drive/MyDrive/data/annotations/trimaps/'  # segmentation map

In [ ]:
image_files = glob(image_dir + '*.jpg')  # 경로 읽기
print(len(image_files))

image_files[:10]   # 경로가 제대로 읽혔는지 확인

In [ ]:
seg_files = glob(seg_dir + '*.png')
print(len(seg_files))

seg_files[:10]   # 경로가 제대로 읽혔는지 확인

In [ ]:
bbox_files = glob(bbox_dir + '*.xml')
print(len(bbox_files))     # image_files, seg_files보다 적음 -> 모든 이미지에 대해 head ROI가 제공되는 것은 아니라는 것

bbox_files[:10]

### Head ROI 시각화

In [ ]:
pip install opencv-python

In [ ]:
import cv2    # image를 읽기 위한 open cv library
import xml.etree.ElementTree as et         # xml 파일을 parsing 하기 위한 library
from matplotlib.patches import Rectangle   # Bounding box를 그리기 위함

cv2.Rectangle(image, pt1 : 시작점 좌표, pt2 : 종료점 좌표, color, thickness, lineType, shift)

In [ ]:
# 임의의 이미지 파일 경로 하나 가져와서 이에 해당하는 bounding box의 경로 찾기
image_path = image_files[80]
bbox_path = image_path.replace(image_dir, bbox_dir).replace('jpg', 'xml')

# image 파일을 open cv로 읽기
# 주의점 : RGB가 아니라 BGR 형식으로 읽힘
image = cv2.imread(image_path)                 # path에 한국어 경로 있으면 안 읽힐 수 있음
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR 형식을 RGB로 바꿔줄 것

# xml파일 parsing 하기
tree = et.parse(bbox_path)

# bounding box 좌표는 object (bounding box) tag 아래 있었음
xmin = float(tree.find('./object/bndbox/xmin').text)
xmax = float(tree.find('./object/bndbox/xmax').text)
ymin = float(tree.find('./object/bndbox/ymin').text)
ymax = float(tree.find('./object/bndbox/ymax').text)

# 가로, 세로 길이
rect_x = xmin
rect_y = ymin
rect_w = xmax - xmin
rect_h = ymax - ymin

# bounding box 객체 만들기 : Head ROI
rect = Rectangle((rect_x, rect_y), rect_w, rect_h, fill=False, color='red')
plt.axes().add_patch(rect)
plt.imshow(image)

plt.show()

### SEG map

- 보통 3가지 채널이 아닌 하나의 채널에 정보가 모두 저장되어 있음 (grayscale)

In [ ]:
image_path = image_files[90]
seg_path = image_path.replace(image_dir, seg_dir).replace('jpg', 'png')

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
seg_map = cv2.imread(seg_path, cv2.IMREAD_GRAYSCALE)  # Grayscale 이미지를 읽어온다는 것 알려줌

# side by side 출력
plt.figure(figsize=(10, 10))
plt.subplot(1,2,1)
plt.imshow(image)
plt.subplot(1,2,2)
plt.imshow(seg_map)

plt.show()

### 이미지 분류 (k-fold)

- validation set으로 index 줘서 fold column 만들어 놓으면 향후 작업하기 편함
- fold별로 데이터가 균등하게 뽑혔는지 항상 주의
  - 요크셔테리어가 전혀 없는 fold 학습하는 모델은 요크셔테리어 분류 못함
  - ★ 방법 : sklearn의 StratifiedKFold 사용

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# fold column 생성 후 -1로 초기화
df['fold'] = -1

for idx, (train, val) in enumerate(kf.split(df), 1):
    print(train, val, len(val))
    df.loc[val, 'fold'] = idx     # validation으로 쓰인 fold index 저장 (1~5)

In [ ]:
df[:5]

print(len(df[df['fold']==1]))  # 첫 번째 fold에 대한 validation set
print(len(df[df['fold']!=1]))  # 첫 번째 fold에 대한 train set

- 강아지와 고양이 품종이 균등하게 분포되어 있는지 시각화로 확인

In [ ]:
value_counts = df[df['fold'] != 5]['id'].value_counts().sort_index()   # 5번째 fold의 training data

plt.bar(range(len(value_counts)), value_counts.values, align='center')
plt.xticks(range(len(value_counts)), value_counts.index.values)
plt.tight_layout()
plt.show()

- StratifiedKFold 사용하기

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

df['fold'] = -1

for idx, (train, val) in enumerate(skf.split(df, df['id']), 1):  # 'id' (품종)을 최대한 균등하게 하겠다
    print(train, val, len(val))
    df.loc[val, 'fold'] = idx

In [ ]:
value_counts = df[df['fold'] != 5]['id'].value_counts().sort_index()   # 5번째 fold의 training data

plt.bar(range(len(value_counts)), value_counts.values, align='center')
plt.xticks(range(len(value_counts)), value_counts.index.values)
plt.tight_layout()
plt.show()

- 추후 학습을 위해 저장

In [ ]:
df.to_csv('kfolds.csv', index=False)

---
# Data Loader

- keras로 data loader를 작성할 때는 sequence를 사용하는데, 일반 generator를 사용하는 것보다 multi-processing에 더 적합하기 때문 (keras 공식문서에서도 sequence 사용을 권장)
- sequence 함수
  - 반드시 있어야 하는 것 : len, getitem
    - len : 한 epoch에 몇 개의 batch가 있는지
    - getitem : 하나의 batch 생성
  - optional : on_epoch_end

### Oxford pet data에 대한 sequence 함수 작성하기

In [ ]:
import math
import random 

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras

In [ ]:
class DataGenerator(keras.utils.Sequence):
    # fold : 5가지 중 어떤 fold를 사용할지
    def __init__(self, batch_size, csv_path, fold, image_size, mode='train', shuffle=True):
        self.batch_size = batch_size
        self.image_size = image_size
        self.fold = fold
        self.mode = mode
        self.shuffle = shuffle
        
        self.df = pd.read_csv(csv_path)
        
        # 필요한 데이터만 남기기
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]  # training
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]  # validation
            
        # Remove invalid files
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'            
        ]
        self.df = self.df[-self.df['file_name']. \
                         isin(invalid_filenames)]
        
        # 학습을 시작하는 첫 epoch에서도 shuffle을 하고 싶은 경우
        self.on_epoch_end()  # 원치 않는 경우 이 부분 코드는 빼기
        
    def __len__(self):
        # 나눠 떨어지지 않는 경우 남는 데이터가 생김
        # -> 올림 필요
        return math.ceil(len(self.df) / self.batch_size)
    
    # idx : 몇 번째 batch를 return 하고 싶은지
    # i.e. 첫 번째 : 0
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        
        batch_x, batch_y = self.get_data(data)
        return np.array(batch_x), np.array(batch_y)
    
    # data 내 모든 이미지와 label return
    def get_data(self, data):
        
        batch_x = []
        batch_y = []
        
        # r : csv 파일의 값들
        for _, r in data.iterrows():
            file_name = r['file_name']
            image = cv2.imread(f'drive/MyDrive/data/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # input image들의 size가 각각 다 다르기 때문에 같은 size로 resize 해줘야 함
            # if not, 하나의 batch로 묶어서 array 만들기 불가
            image = cv2.resize(image, (self.image_size, self.image_size))
            
            # rescale : 0~255 px -> 0~1
            image = image / 255.
            
            # 개와 고양이를 분류하는 이진분류
            # 현재 데이터 : 고양이 1, 개 2 -> -1 -> 고양이 0, 개 1
            label = int(r['species']) - 1
            
            batch_x.append(image)
            batch_y.append(label)
            
        return batch_x, batch_y
            
    # callback 함수
    # 매 epoch이 끝날 때마다 도는 함수
    # ex) shuffle 하는 함수를 넣으면 한 epoch이 끝날 때마다 데이터 자동 shuffle
    def on_epoch_end(self):
        if self.shuffle:
            # frac : 몇 퍼센트의 데이터를 shuffle 할지 (1 -> 100% : 전체 데이터)
            # drop=True : 기존 인덱스를 버리고 재배열
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [ ]:
csv_path = 'kfolds.csv'
train_generator = DataGenerator(
    batch_size=9,
    csv_path=csv_path,
    fold=1,
    image_size=256,  # 256×256
    mode='train',
    shuffle=True
)

데이터 개수 확인
- 전체 데이터 : 7,390개
- 5-fold + train mode -> 전체 데이터의 80% 정도가 있어야 함 (5.8천 여 개)

In [ ]:
print(len(train_generator))
print(len(train_generator) * 9)   # 9 : batch size

data generator가 이미지를 제대로 load 하는 지 확인

In [ ]:
class_name = ['Cat', 'Dog']  # 고양이 0, 강아지 1

for batch in train_generator:
    # X: image, y: label
    X, y = batch
    plt.figure(figsize=(10, 10))
    
    for i in range(9):   # batch size가 9개 였으니 9개 이미지 살펴볼 예정
        ax = plt.subplot(3, 3, i+1)   # 총 3*3개 plot, i+1 번째에 그리기
        plt.imshow(X[i])
        plt.title(class_name[y[i]])
        plt.axis('off')
    
    # 첫 번째 batch만 확인할 예정
    break

In [ ]:
for batch in train_generator:
    # X: image, y: label
    X, y = batch
    print(y)
    break

---
# Model 구현

### Sequential 모델 구현

- simple한 모델들만 구현 가능
- sequential class 단독으로 사용하기 보단, model의 block 단위를 간단하게 구현할 때 사용

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
def get_sequential_model(input_shape):
    
    # layer를 순서대로 쌓기
    model = keras.Sequential(
        [
            layers.Input(input_shape),  # input
            
            # 1st Conv block
            # convolution을 적용하면 image size가 2px 씩 작아짐
            # 여러 번 거치면 완전 작아짐
            # 이를 방지하기 위해 padding 사용 (same : input과 output의 크기가 일치)
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),  # filter size 64, kerner size 3*3
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            
            # CNN에서 많이 쓰이는 heuristic
            # MaxPooling 거쳐서 이미지 사이즈가 반으로 감소
            # 그만큼 다음 layer에서 filter 사이즈를 키움 (64 -> 128)
            # VGG 때부터 많이 쓰인 방식
            
            # 2nd Conv block
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            
            # MaxPooling
            # - 각각의 filter에 대해 주어진 kernel 안에서 최대값 return -> kernel 사이즈가 작아짐
            # GlobalMaxPooling
            # - 한 필터에서 하나의 값 가져옴 -> 128개의 필더가 있으면 총 128개 값 return
            #   각 필터의 대표값을 뽑을 수 있고 parameter 수가 감소하니 연산속도 빨라짐
            
            # Classifier
            layers.GlobalMaxPool2D(),
            layers.Dense(128, activation='relu'),
            layers.Dense(1, activation='sigmoid')  # 이진분류 (개, 고양이)            
        ]
    )
    
    return model

input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.summary()

### Functional API 모델 구현

- keras 공식 모델이 저장되어 있는 keras applications에 있는 모델들
- Sequential과 달리 multi input/output, layer sharing 등 특별한 제약 없이 다양한 기능 사용 가능

In [ ]:
def get_functional_model(input_shape):
    
    # Sequential과 달리 사용자가 직접 모든 layer의
    # 결과를 저장하고 다음 레이어에 넘겨줘야 함
    inputs = keras.Input(input_shape)
    
    # 1st Conv block
    x = layers.Conv2D(64, 3, strides=1, activation='relu', padding='same')(inputs)
    x = layers.Conv2D(64, 3, strides=1, activation='relu', padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    # 2nd Conv block
    x = layers.Conv2D(128, 3, strides=1, activation='relu', padding='same')(x)
    x = layers.Conv2D(128, 3, strides=1, activation='relu', padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    # Classifier
    x = layers.GlobalMaxPool2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)  # 이진분류 (개, 고양이)
    
    model = keras.Model(inputs, outputs)
    
    return model

input_shape = (256, 256, 3)
model = get_functional_model(input_shape)

model.summary()

### Model Subclassing

- pytorch 경험자들은 이 방법이 가장 수월할 듯
- keras의 model class를 상속받아서 필요한 method를 override
  - functional API와 달리, model.fit, model.evaluate, model.predict를 override 해서 customize 가능
  - override 할 필요가 없는 경우 굳이 model subclassing보다는 functional API 사용하는 것으로 충분

In [ ]:
class SimpleCNN(keras.Model):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # keras에서는 하나의 block을 하나의 layer로 생각함
        # -> 모델에는 3가지 layer만 있는 것처럼 인지됨
        # -> 나중에 model.summary() 해보면 결과가 다른 모델 대비 간단한 이유
        self.conv_block_1 = keras.Sequential(
            [
                layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
                layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.5)          
            ], name='conv_block_1'
        )
        
        self.conv_block_2 = keras.Sequential(
            [
                layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
                layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.3)
            ], name='conv_block_2'
        )
        
        self.classifier = keras.Sequential(
            [
                layers.GlobalMaxPool2D(),
                layers.Dense(128, activation='relu'),
                layers.Dense(1, activation='sigmoid')  # 이진분류 (개, 고양이)                   
            ], name='classifier'
        )
    
    # 모델 호출 시 작동할 함수
    # 위에서 작성한 layer들이 어떤 순서로 불려야 하는지 모델 순서 정하기
    def call(self, input_tensor, training=False):
        x = self.conv_block_1(input_tensor)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x

In [ ]:
# None : batch axis 부분
# -> batch 값은 모델의 구조를 정하는데 중요하지 않아 일반적으로 적용
input_shape = (None, 256, 256, 3)

model = SimpleCNN()
model.build(input_shape)
model.summary()

## Model compiling

- keras에서 학습에 사용할 optimizer, loss, 평가에 사용할 metric을 정해주는 과정

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)

---
# Summary

In [ ]:
import os
import math

import cv2
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            # Input
            layers.Input(input_shape),

            # 1st Conv block
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.5),

            # 2nd Conv block
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
        
            # Classfier
            layers.GlobalMaxPool2D(),
            layers.Dense(128, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ]
    )

    return model

input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)

model.summary()

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, image_size,
                 fold, mode='train', shuffle=True):
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.fold = fold
        self.mode = mode
        
        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
        
        #### Remove invalid files
        #### https://github.com/tensorflow/models/issues/3134
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'
        ]
        self.df = self.df[~self.df['file_name']. \
                          isin(invalid_filenames)]
        self.on_epoch_end()
            
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
        
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        
        batch_x, batch_y = self.get_data(data)
        
        return np.array(batch_x), np.array(batch_y)
        
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for _, r in data.iterrows():
            file_name = r['file_name']
            
            image = cv2.imread(f'drive/MyDrive/data/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            image = cv2.resize(image, (self.image_size, self.image_size))
            image = image / 255.
            
            label = int(r['species']) - 1
            
            batch_x.append(image)
            batch_y.append(label)
        
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [ ]:
csv_path = 'kfolds.csv'

train_generator = DataGenerator(
    fold=1,
    mode='train',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

valid_generator = DataGenerator(
    fold=1,
    mode='val',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

## Callback functions

- 특정 상황에서 호출된 함수
  - on_train_begin : 학습이 시작될 때 호출
  - on_epoch_end : 매 epoch이 끝날 때마다 호출
- 아래에서 자주 쓰이는 callback 함수 3가지를 보고자 함
- 만약 early stopping (patience: 3)과 reduce on plateau (patience: 10)를 같이 쓰면, 어차피 3일 때 early stopping이 될 것이기 때문에 실제로 reduce on plateau가 불리는 일은 없음

### Early Stopping

- 성능이 나아지지 않으면 학습 중단
- 아래 코드 : validation loss가 3 epoch 동안 줄어들지(min) 않으면 학습 멈추기

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1,
    mode="min",
    restore_best_weights=False
)

### Reduce on plateau

- 지정한 epoch동안 지정한 metric 성능이 나아지지 않으면 learning rate를 줄임
- 아래 코드
  - validation loss이 10 epoch 동안 작아지지 않으면 learning rate를 0.1배로 줄이자
  - 하지만 learning rate는 0.001보다는 작아지면 안 된다

In [ ]:
reduce_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=10,
    verbose=1,
    mode='min',
    min_lr=0.001
)

### 모델 저장

In [ ]:
filepath = 'drive/MyDrive/data/{epoch:02d}-{val_loss:.2f}.hdf5'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='val_loss',  # validation loss가 작아질 때마다 저장
    verbose=1,
    save_best_only=True,     # best 외 불필요한 건 저장X
    save_weights_only=False, # weight 파일만 따로 저장할지
    mode='min'
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    callbacks=[
        early_stopping,
        reduce_on_plateau,
        model_checkpoint
    ],
    verbose=1
)

- training/validation의 loss, accuracy 확인

In [ ]:
history.history

### 시각화

In [ ]:
import matplotlib.pyplot as plt

history = history.history

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='train')
plt.plot(history['val_accuracy'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title("Accuracy")
plt.show()

---
# Data Augmentation

- 데이터 양을 늘리기 위해 원본 이미지를 여러 가지 방식으로 변환하는 것
- 결함이 있는 데이터에 대해서도 robust한 모델을 만들기 위함
- 3rd party library 사용함
  - 많은 사람들이 사용하는 것 추천 - Albumentations : made by kaggle masters

- 이 코드로 albumentations install 하니까 CoarseDropout을 인지하지 못하는 문제는 사라졌지만 cv2가 import되지 않는 문제 발생
- cv2를 uninstall 했다가 다시 install 하니까 모든 문제 해결됨

In [ ]:
pip install git+https://github.com/albumentations-team/albumentations.git

In [ ]:
pip uninstall opencv-python

In [ ]:
pip install opencv-python

In [ ]:
import os
import math
import random

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

import albumentations as A

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
# augmentation : 보통 training단계에서 많이 사용됨

class Augmentation:
    def __init__(self, size, mode='train'):
        if mode == 'train':
            # Declare an augmentation pipeline
            # 여러 번 호출할 예정이라 transform이 아닌 self.transform으로 저장함
            self.transform = A.Compose([
                # A.RandomCrop(width=256, height=256),  # resize를 외부에서 따로 할 예정
                
                # 좌우반전 (test 이미지에서 상하반전은 별로 없지만 좌우반전된 이미지는 많이 있을 것 같아서 추가함)
                # p : 해당 변화를 적용할 확률
                A.HorizontalFlip(p=0.5),

                # Shift : 이미지를 상하좌우로 움직이는 것
                # Scale : 이미지를 확대/축소하는 것
                # Rotate : 이미지를 회전하는 것
                A.ShiftScaleRotate(
                    p=0.5,
                    shift_limit=0.05,  # 이미지의 가로 길이가 최대 몇% 넘어가도 되는 지
                    scale_limit=0.05,  # 이미지를 최대 몇% 확대/축소할지
                    rotate_limit=15,
                ),

                # 이미지에 구멍을 뚫는 것
                A.CoarseDropout(
                    p=0.5,
                    max_holes=8,  # 최대 구멍 개수
                    max_height=int(0.1 * size),  # 가로 최대 길이 : 이미지의 10%
                    max_width=int(0.1 * size),
                ),

                A.RandomBrightnessContrast(p=0.2),
            ])

    def __call__(self, **kwargs):
        if self.transform:   # train mode인 경우
            augmented = self.transform(**kwargs)
            img = augmented['image']
            return img

- DataGenerator 코드에 Augmentation 코드 추가하기

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, image_size,
                 fold, mode='train', shuffle=True):
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.fold = fold
        self.mode = mode
        
        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
        
        #### Remove invalid files
        #### https://github.com/tensorflow/models/issues/3134
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'
        ]
        self.df = self.df[~self.df['file_name']. \
                          isin(invalid_filenames)]

        self.transform = Augmentation(image_size, mode)  ##########

        self.on_epoch_end()
            
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
        
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        
        batch_x, batch_y = self.get_data(data)
        
        return np.array(batch_x), np.array(batch_y)
        
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for _, r in data.iterrows():
            file_name = r['file_name']
            
            image = cv2.imread(f'drive/MyDrive/data/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            image = cv2.resize(image, (self.image_size, self.image_size))

            if self.mode == 'train':   ##########
                # augmentation 중에는 image가 uint8인 경우에만 적용되는 것들이 있음
                # -> image를 먼저 uint8로 변환하기
                image = image.astype('uint8')
                image = self.transform(image=image)

            image = image.astype('float32')
            image = image / 255.
            
            label = int(r['species']) - 1
            
            batch_x.append(image)
            batch_y.append(label)
        
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [ ]:
csv_path = 'kfolds.csv'

train_generator =  DataGenerator(
    fold=1,
    mode='train',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

valid_generator =  DataGenerator(
    fold=1,
    mode='val',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

- CoarseDropout이 적용된 결과 0.5%의 확률로 특정 이미지에 hole들이 뚫려있음
- 랜덤하게 적용되기 때문에 아래 코드 다시 돌려보면 CoarseDropout 적용된 이미지가 계속 달라짐
- 회전된 이미지도 볼 수 있음

In [ ]:
class_name = ['Cat', 'Dog']

for batch in train_generator:
    X, y = batch
    plt.figure(figsize=(10,10))

    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(X[i])
        plt.title(class_name[y[i]])
        plt.axis('off')
    break

- 학습에 적용

In [ ]:
def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            # Input
            layers.Input(input_shape),

            # 1st Conv block
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.5),

            # 2nd Conv block
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
        
            # Classfier
            layers.GlobalMaxPool2D(),
            layers.Dense(128, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ]
    )

    return model

input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)

model.summary()

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1
)

---
# Transfer Learning

- 배경
  - dataset의 고양이와 개 비율 = 2:5
    - 모든 data를 강아지로 찍으면 성능이 70% 정도
  - 위에서 만든 모델의 성능 : validation data 70% accuracy
  - 즉, 모델 학습을 했음에도 불구하고 찍었을 때와 큰 차이가 없음
  - 그냥 학습을 했을 때와 data augmentation 적용했을 때도 최종 모델 성능 차이가 거의 없었음
  - 모델 바꿀 필요 있음 -> Transfer Learning
- Transfer Learning
  - 특정 분야에서 학습된 모델을 유사하거나 전혀 새로운 분야에 사용하는 것
    - 수십만 장의 image 학습하고 유의미한 feature 추출
    - 이를 oxford pet data 분류에 사용
  - 학습데이터가 적을 때 유용 + 학습 속도, 최종 결과도 모두 좋음 -> 항상 모델을 pre train 하는 것 추천
- pre trained된 모델을 가져올 수 있는 곳
  - https://github.com/keras-team/keras-applications
  - https://keras.io/api/applications/
  - VGG, ResNet, EfficientNet, etc.
  - 이 중 image net 분류에 대해 성능(Top-1 accuracy 부분)이 좋게 적혀 있는 모델일수록 내가 하려는 task에 쓸 때도 성능이 좋음
  - 주의 : 모델의 Size가 클수록 학습속도 느림
    - 느려도 상관 없는지 vs real-time이 중요한지
  - accuracy와 size의 trade off 잘 고려해서 선택할 것

In [ ]:
import os
import math

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.applications import EfficientNetB0

import albumentations as A

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

### EfficientNet

- 모델을 불러서 내가 사용하려는 것에 맞게 변환하기
- `model.summary()` 결과
  - efficientnetb0가 하나의 layer처럼 사용됨
  - 위에 쓴 모델은 parameter 수가 27만 개 정도 됐는데 parameter 수도 훨씬 많아짐 (400만)
  - 마지막 dense : 실제 분류하는 layer

In [ ]:
def get_model(input_shape):

    inputs = keras.Input(input_shape)

    # Feature extract
    base_model = EfficientNetB0(
        input_shape=input_shape,

        # 만약 아무 것도 안 쓰면 transfer을 사용하는 게 아니라 모델 구현체만 가져다 학습에사용하겠다는 의미
        weights='imagenet',   # imagenet의 pre trained된 weight값 사용
        
        # github 링크에서 모델 코드 보면 if include_top: 부분이 있음
        # 해당 모델이 image를 분류하기 위해 사용한 layer들
        # = image에서 feature를 추출하는 layer 외에 그 feature를 가지고 분류하는 layer 부분임
        # = 내가 사용하려는 task랑 크게 관련은 없음 + 무거움
        # -> 이걸 가져다 쓰지 않고 (False) 내 task에 맞게 따로 구현하고자 함
        include_top=False,

        pooling='avg'  # Gloabl average pooling
    )

    x = base_model(inputs)  # image의 feature 추출
    outputs = layers.Dense(1, activation='sigmoid')(x)  # 이진분류
    model = keras.Model(inputs, outputs)

    return model

input_shape = (256, 256, 3)
model = get_model(input_shape)

# Transfer learning 할 때는 보통 사용하는 learning rate(0.001)보다 작은 값 사용
adam = keras.optimizers.Adam(lr=0.0001)

model.compile(
    optimizer=adam,
    loss='binary_crossentropy',
    metrics='accuracy'
)

model.summary()

In [ ]:
class Augmentation:
    def __init__(self, size, mode='train'):
        if mode == 'train':
            self.transform = A.Compose([
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(
                    p=0.5,
                    shift_limit=0.05,  # 이미지의 가로 길이가 최대 몇% 넘어가도 되는 지
                    scale_limit=0.05,  # 이미지를 최대 몇% 확대/축소할지
                    rotate_limit=15,
                ),

                # 이미지에 구멍을 뚫는 것
                A.CoarseDropout(
                    p=0.5,
                    max_holes=8,  # 최대 구멍 개수
                    max_height=int(0.1 * size),  # 가로 최대 길이 : 이미지의 10%
                    max_width=int(0.1 * size),
                ),

                A.RandomBrightnessContrast(p=0.2),
            ])

    def __call__(self, **kwargs):
        if self.transform:   # train mode인 경우
            augmented = self.transform(**kwargs)
            img = augmented['image']
            return img

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, image_size,
                 fold, mode='train', shuffle=True):
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.fold = fold
        self.mode = mode
        
        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
        
        #### Remove invalid files
        #### https://github.com/tensorflow/models/issues/3134
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'
        ]
        self.df = self.df[~self.df['file_name']. \
                          isin(invalid_filenames)]

        self.transform = Augmentation(image_size, mode)

        self.on_epoch_end()
            
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
        
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        
        batch_x, batch_y = self.get_data(data)
        
        return np.array(batch_x), np.array(batch_y)
        
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for _, r in data.iterrows():
            file_name = r['file_name']
            
            image = cv2.imread(f'drive/MyDrive/data/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            image = cv2.resize(image, (self.image_size, self.image_size))

            if self.mode == 'train':
                image = image.astype('uint8')
                image = self.transform(image=image)

            image = image.astype('float32')
            image = image / 255.
            
            label = int(r['species']) - 1
            
            batch_x.append(image)
            batch_y.append(label)
        
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [ ]:
csv_path = 'kfolds.csv'

train_generator =  DataGenerator(
    fold=1,
    mode='train',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

valid_generator =  DataGenerator(
    fold=1,
    mode='val',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

- 성능이 굉장히 좋아짐

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt

history = history.history

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='train')
plt.plot(history['val_accuracy'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title("Accuracy")
plt.show()

---
# TF Hub

- 배경
  - keras applications는 마지막 update가 몇 년 전
  - efficientNet 이후에 나온 모델들에 대해서는 update가 안 됨
- TF Hub
  - https://tfhub.dev/
  - tensorflow에서 공식적으로 운영
  - 최신 모델 (weight 포함) 지속 update 됨
  - keras에는 image만 있는 것과 달리 text, video, audio 모델까지 포함
- 방법
  - problem domains에서 image 클릭
  - filter 적용 : TF2 (version) + Finetunable 켜기
  - efficientNet v2 검색
    - efficientNet 이후에 나온 것
    - 이걸 사용해서 학습하면 많이 update가 되었기 때문에 편리함
  - 원하는 것 클릭 : imagenet/efficientnet_v2_imagenet1k_b0/feature_vector
    - 기존 모델들보다 성능 좋음
    - Usage 부분에 사용방법 코드 있음
    - 그대로 복사해서 가져오기

In [ ]:
import os
import math

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.applications import EfficientNetB0

import albumentations as A

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

- 가장 최신 모델이라고 할 수 있는 efficientnet_v2를 tensorflow hub에서 가져와서 내 task 학습에 적용

In [ ]:
import tensorflow_hub as hub

# Usage 코드
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2",  # 기존 구현된 모델
                   trainable=True),  # base 모델도 학습이 가능하게 설정
    tf.keras.layers.Dense(1, activation='sigmoid')  # 내 모델의 classification layer 붙이기 (binary)
])

model.build([None, 256, 256, 3])  # Batch input shape

adam = keras.optimizers.Adam(lr=0.0001)
model.compile(
    optimizer=adam,
    loss='binary_crossentropy',
    metrics='accuracy'
)

model.summary()

In [ ]:
class Augmentation:
    def __init__(self, size, mode='train'):
        if mode == 'train':
            self.transform = A.Compose([
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(
                    p=0.5,
                    shift_limit=0.05,  # 이미지의 가로 길이가 최대 몇% 넘어가도 되는 지
                    scale_limit=0.05,  # 이미지를 최대 몇% 확대/축소할지
                    rotate_limit=15,
                ),

                # 이미지에 구멍을 뚫는 것
                A.CoarseDropout(
                    p=0.5,
                    max_holes=8,  # 최대 구멍 개수
                    max_height=int(0.1 * size),  # 가로 최대 길이 : 이미지의 10%
                    max_width=int(0.1 * size),
                ),

                A.RandomBrightnessContrast(p=0.2),
            ])

    def __call__(self, **kwargs):
        if self.transform:   # train mode인 경우
            augmented = self.transform(**kwargs)
            img = augmented['image']
            return img

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, image_size,
                 fold, mode='train', shuffle=True):
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.fold = fold
        self.mode = mode
        
        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
        
        #### Remove invalid files
        #### https://github.com/tensorflow/models/issues/3134
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'
        ]
        self.df = self.df[~self.df['file_name']. \
                          isin(invalid_filenames)]

        self.transform = Augmentation(image_size, mode)

        self.on_epoch_end()
            
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
        
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        
        batch_x, batch_y = self.get_data(data)
        
        return np.array(batch_x), np.array(batch_y)
        
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for _, r in data.iterrows():
            file_name = r['file_name']
            
            image = cv2.imread(f'drive/MyDrive/data/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            image = cv2.resize(image, (self.image_size, self.image_size))

            if self.mode == 'train':
                image = image.astype('uint8')
                image = self.transform(image=image)

            image = image.astype('float32')
            image = image / 255.
            
            label = int(r['species']) - 1
            
            batch_x.append(image)
            batch_y.append(label)
        
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

- 모델 사이즈가 커졌기 때문에 batch size를 절반 정도로 줄임

In [ ]:
csv_path = 'kfolds.csv'

train_generator =  DataGenerator(
    fold=1,
    mode='train',
    csv_path=csv_path,
    batch_size=64,
    image_size=256,
    shuffle=True
)

valid_generator =  DataGenerator(
    fold=1,
    mode='val',
    csv_path=csv_path,
    batch_size=64,
    image_size=256,
    shuffle=True
)

- efficientNet v2가 차이가 없는 이유
  - 지금 우리가 사용하는 task가 비교적 쉬워서 성능 차이가 별로 없지만
  - 다른 이미지 분류에 사용하면 b0보다 v2가 성능이 좋음

In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt

history = history.history

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='train')
plt.plot(history['val_accuracy'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title("Accuracy")
plt.show()

---
# Multiclass Classification

- Oxford pet dataset
  - 총 37 종류의 개와 고양이를 구분하는 label 있음 (ID 컬럼)
- 해결책은 이진분류와 크게 다르지 않음
  - 위에서 사용한 efficientNet B0 코드를 그대로 활용하고자 함

In [ ]:
import os
import math

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.applications import EfficientNetB0

import albumentations as A

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

- 각 class에 속하는 data는 200개 씩 균등하게 분포되어 있음

In [ ]:
csv_path = 'kfolds.csv'
df = pd.read_csv(csv_path)
np.unique(df['id'])

value_counts = df['id'].value_counts().sort_index()

plt.figure(figsize=(10, 5))
plt.bar(range(len(value_counts)), value_counts.values)
plt.xticks(range(len(value_counts)), value_counts.index.values)
plt.tight_layout()
plt.show()

- multi-class classification에 맞게 binary classification 수정

In [ ]:
def get_model(input_shape):

    inputs = keras.Input(input_shape)

    # Feature extract
    base_model = EfficientNetB0(
        input_shape=input_shape,
        weights='imagenet',
        include_top=False,
        pooling='avg'
    )

    x = base_model(inputs)
    # softmax : 각 class에 대한 확률
    outputs = layers.Dense(37, activation='softmax')(x)  # multi-class에 맞게 output, activation 변경
    model = keras.Model(inputs, outputs)

    return model

input_shape = (256, 256, 3)
model = get_model(input_shape)

adam = keras.optimizers.Adam(lr=0.0001)

model.compile(
    optimizer=adam,
    # 원래 ouput이 37개니까 이에 맞게 label도 one-hot encoding을 해 줘야 하지만
    # sparse_categorical_crossentropy를 사용하면 one-hot encoding 필요 없음
    # label index만 넘겨서 모델이 학습하게 하기 때문
    loss='sparse_categorical_crossentropy', # multi-class에 맞게 변경
    metrics='accuracy'
)

model.summary()

- Augmentation : 수정 필요 없음

In [ ]:
class Augmentation:
    def __init__(self, size, mode='train'):
        if mode == 'train':
            self.transform = A.Compose([
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(
                    p=0.5,
                    shift_limit=0.05,  # 이미지의 가로 길이가 최대 몇% 넘어가도 되는 지
                    scale_limit=0.05,  # 이미지를 최대 몇% 확대/축소할지
                    rotate_limit=15,
                ),

                # 이미지에 구멍을 뚫는 것
                A.CoarseDropout(
                    p=0.5,
                    max_holes=8,  # 최대 구멍 개수
                    max_height=int(0.1 * size),  # 가로 최대 길이 : 이미지의 10%
                    max_width=int(0.1 * size),
                ),

                A.RandomBrightnessContrast(p=0.2),
            ])

    def __call__(self, **kwargs):
        if self.transform:   # train mode인 경우
            augmented = self.transform(**kwargs)
            img = augmented['image']
            return img

- DataGenerator 일부 수정 필요 : label을 return 하는 부분

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, image_size,
                 fold, mode='train', shuffle=True):
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.fold = fold
        self.mode = mode
        
        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
        
        #### Remove invalid files
        #### https://github.com/tensorflow/models/issues/3134
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'
        ]
        self.df = self.df[~self.df['file_name']. \
                          isin(invalid_filenames)]

        self.transform = Augmentation(image_size, mode)

        self.on_epoch_end()
            
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
        
    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        
        batch_x, batch_y = self.get_data(data)
        
        return np.array(batch_x), np.array(batch_y)
        
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for _, r in data.iterrows():
            file_name = r['file_name']
            
            image = cv2.imread(f'drive/MyDrive/data/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            image = cv2.resize(image, (self.image_size, self.image_size))

            if self.mode == 'train':
                image = image.astype('uint8')
                image = self.transform(image=image)

            image = image.astype('float32')
            image = image / 255.
            
            # sparse_categorical_crossentropy를 사용하기 때문에
            # 따로 one-hot encoding 변환 없이 기존처럼 id의 index만 넘기면 됨
            # 1~37로 되어있으니 -1을 해서 0~36으로 변환
            label = int(r['id']) - 1  #####
            
            batch_x.append(image)
            batch_y.append(label)
        
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [ ]:
csv_path = 'kfolds.csv'

train_generator =  DataGenerator(
    fold=1,
    mode='train',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

valid_generator =  DataGenerator(
    fold=1,
    mode='val',
    csv_path=csv_path,
    batch_size=128,
    image_size=256,
    shuffle=True
)

- 10 epoch로 학습했을 때 이진분류 때보다는 성능이 약간 낮음
- train과 validation 사이 차이도 커짐
- 그래도 매 epoch마다 train, validation 모두 accuracy가 증가하고 있기 때문에 epoch 개수를 늘리면 성능이 더 좋아질 가능성이 큼


In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt

history = history.history

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='train')
plt.plot(history['val_loss'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title("Loss")

plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='train')
plt.plot(history['val_accuracy'], label='val')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title("Accuracy")
plt.show()